**Data Preprocessing**- here we will use the raw data to clean and mainpulate raw data to makeit useful for further use.
> Data is first merged in a (merged_labels)
> Data is then preprocessed and saved into (filtered_data)



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

icbhi_metadata = pd.read_csv('/content/drive/MyDrive/DL project/labels/ICBHI_Metadata_Updated.xlsx - Sheet1.csv')  # Replace with your file path
fraiwan_metadata = pd.read_csv('/content/drive/MyDrive/DL project/labels/Fraiwan_Metadata (1).xlsx - Sheet1.csv')  # Replace with your file path

In [ ]:
import librosa

def load_audio(file_path):
  audio, sr = librosa.load(file_path)
  return audio, sr

# Example usage:
icbhi_audio, sr = load_audio('/content/drive/MyDrive/DL project/ICBHI_audio_files/101_1b1_Pr_sc_Meditron.wav')  # Replace with your file path
fraiwan_audio, sr = load_audio('/content/drive/MyDrive/DL project/Fraiwan_Audio_Files/BP100_N,N,P R M,70,F.wav')  # Replace with your file path
print(icbhi_audio, sr)
print(fraiwan_audio, sr)

[-0.02719397 -0.0398677  -0.03535346 ... -0.04244613 -0.04132774
 -0.04538753] 22050
[ 1.5497208e-06  2.1047890e-06  2.3059547e-06 ... -9.0670139e-03
 -5.3604916e-03 -2.0680986e-03] 22050


Imagine the audio waveform as a curve that goes up and down.
The numbers in the arrays represent the height of the curve at different points.
The sample rate tells you how many points on the curve were measured every second i,sr is 22050Hz

# Merge labels into one single file

In [ ]:
# Merge the metadata
merged_metadata = pd.concat([icbhi_metadata, fraiwan_metadata], ignore_index=True)

# Save the merged metadata (optional)
merged_metadata.to_csv('/content/drive/MyDrive/DL project/labels/merged_label.csv', index=False)  # Replace with your desired file path

In [ ]:
merged_metadata.()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1256 entries, 0 to 1255
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Audio File Name  1256 non-null   object 
 1   Age              1250 non-null   float64
 2   Gender           1250 non-null   object 
 3   Location         1256 non-null   object 
 4   Sound Type       1256 non-null   object 
 5   Condition        1256 non-null   object 
dtypes: float64(1), object(5)
memory usage: 59.0+ KB


# Meta data pre-processing in merged file
1. See what kind of data is there in (merged_label)
2. Lowercase everything, keep relevant classes
3. replace bron with bronchitis
4. Reset index and save data as (filetred_label)


In [ ]:
unique_labels_condition = merged_metadata['Condition'].unique()  # Assuming 'diagnosis' is the label column
print(unique_labels_condition)
unique_labels_sound = merged_metadata['Sound Type'].unique()  # Assuming 'diagnosis' is the label column
print(unique_labels_sound)

['COPD' 'URTI' 'Asthma' 'LRTI' 'Bronchiectasis' 'Healthy' 'Pneumonia'
 'Bronchiolitis' 'N' 'Lung Fibrosis' 'copd' 'asthma' 'BRON'
 'heart failure' 'Asthma and lung fibrosis' 'Heart Failure' 'pneumonia'
 'Heart Failure + COPD' 'Heart Failure + Lung Fibrosis '
 'Plueral Effusion']
['normal' 'wheeze' 'crackle' 'wheeze, crackle' 'E W' 'I E W' 'N' 'Crep'
 'C' 'I C E W' 'I C' 'Bronchial' 'I C B']


In [ ]:
# Convert 'Condition' column to lowercase
merged_metadata['Condition'] = merged_metadata['Condition'].str.lower()

# unique conditions
unique_labels_condition = merged_metadata['Condition'].unique()  # Assuming 'diagnosis' is the label column
print(unique_labels_condition)

# Define the classes to keep (also in lowercase)
classes_to_keep = ['asthma', 'bronchiectasis', 'bronchiolitis', 'bron', 'copd',
                     'lung fibrosis', 'plueral effusion', 'pneumonia', 'urti', 'healthy', 'n']

# Filter the DataFrame
filtered_metadata = merged_metadata[merged_metadata['Condition'].isin(classes_to_keep)]

filtered_metadata['Condition'] = filtered_metadata['Condition'].replace('n', 'healthy')

unique_label_conditions = ['copd', 'urti', 'asthma', 'bronchiectasis', 'healthy', 'pneumonia',
                           'bronchiolitis', 'lung fibrosis', 'bron', 'plueral effusion']

# Replace 'bron' with 'bronchitis'
unique_label_conditions = [label if label != 'bron' else 'bronchitis' for label in unique_label_conditions]

print(unique_label_conditions) # To verify the change

filtered_metadata['Condition'] = filtered_metadata['Condition'].replace('bron', 'bronchitis')

# (Optional) Reset the index
filtered_metadata = filtered_metadata.reset_index(drop=True)

# (Optional) Save the filtered metadata
filtered_metadata.to_csv('/content/drive/MyDrive/DL project/labels/filtered_label.csv', index=False)

['copd' 'urti' 'asthma' 'lrti' 'bronchiectasis' 'healthy' 'pneumonia'
 'bronchiolitis' 'n' 'lung fibrosis' 'bron' 'heart failure'
 'asthma and lung fibrosis' 'heart failure + copd'
 'heart failure + lung fibrosis ' 'plueral effusion']
['copd', 'urti', 'asthma', 'bronchiectasis', 'healthy', 'pneumonia', 'bronchiolitis', 'lung fibrosis', 'bronchitis', 'plueral effusion']


<ipython-input-32-9739129e1a87>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_metadata['Condition'] = filtered_metadata['Condition'].replace('n', 'healthy')
<ipython-input-32-9739129e1a87>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_metadata['Condition'] = filtered_metadata['Condition'].replace('bron', 'bronchitis')


In [ ]:
from IPython.display import display

display(filtered_metadata)

# Set options to display all rows and columns
pd.set_option("display.max_rows", None, "display.max_columns", None)

# Now, simply print the DataFrame:
filtered_metadata


,Audio File Name,Age,Gender,Location,Sound Type,Condition
0,104_1b1,70.00,F,Lr,normal,copd
1,105_1b1,7.00,F,Tc,normal,urti
2,103_2b2,70.00,F,Ar,wheeze,asthma
3,104_1b1,70.00,F,Ll,normal,copd
4,104_1b1,70.00,F,Ar,normal,copd
5,101_1b1,3.00,F,Al,normal,urti
6,104_1b1,70.00,F,Al,normal,copd
7,104_1b1,70.00,F,Pr,normal,copd
8,106_2b1,73.00,F,Pr,crackle,copd
9,107_2b5,75.00,F,Pr,crackle,copd


,Audio File Name,Age,Gender,Location,Sound Type,Condition
0,104_1b1,70.00,F,Lr,normal,copd
1,105_1b1,7.00,F,Tc,normal,urti
2,103_2b2,70.00,F,Ar,wheeze,asthma
3,104_1b1,70.00,F,Ll,normal,copd
4,104_1b1,70.00,F,Ar,normal,copd
5,101_1b1,3.00,F,Al,normal,urti
6,104_1b1,70.00,F,Al,normal,copd
7,104_1b1,70.00,F,Pr,normal,copd
8,106_2b1,73.00,F,Pr,crackle,copd
9,107_2b5,75.00,F,Pr,crackle,copd


In [ ]:
filtered_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1188 entries, 0 to 1187
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Audio File Name  1188 non-null   object 
 1   Age              1182 non-null   float64
 2   Gender           1182 non-null   object 
 3   Location         1188 non-null   object 
 4   Sound Type       1188 non-null   object 
 5   Condition        1188 non-null   object 
dtypes: float64(1), object(5)
memory usage: 55.8+ KB


In [ ]:
unique_labels_condition = filtered_metadata['Condition'].unique()  # Assuming 'diagnosis' is the label column
print(unique_labels_condition)

['copd' 'urti' 'asthma' 'bronchiectasis' 'healthy' 'pneumonia'
 'bronchiolitis' 'lung fibrosis' 'bronchitis' 'plueral effusion']


Raw data info

# View number of classes in initial data

In [ ]:
condition_counts = filtered_metadata['Condition'].value_counts()
print(condition_counts)

Condition
copd                820
healthy             140
asthma               97
pneumonia            52
urti                 23
bronchiectasis       16
bronchiolitis        13
lung fibrosis        12
bronchitis            9
plueral effusion      6
Name: count, dtype: int64
